<h1 align="center"><font color="yellow">Pytorch: otimizado</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

Até este ponto, atualizamos os pesos de nossos modelos alterando manualmente os Tensores que contêm parâmetros apreensíveis com `torch.no_grad()`. Isso não é um fardo enorme para algoritmos de otimização simples como `descida de gradiente estocástico`, mas na prática frequentemente treinamos redes neurais usando otimizadores mais sofisticados como `AdaGrad`, `RMSProp`, `Adam`, etc.

O pacote `optim no PyTorch` abstrai a ideia de um algoritmo de otimização e fornece implementações de algoritmos de otimização comumente usados.

Neste exemplo vamos usar o pacote nn para definir nosso modelo como antes, mas vamos otimizar o modelo usando o algoritmo `RMSprop` fornecido pelo pacote optim:

In [1]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)


In [3]:
xx

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [2]:
xx.shape

torch.Size([2000, 3])

In [4]:
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')


<font color="orange">Use o pacote `optim` para definir um Optimizer que atualizará os `pesos` do modelo para nós. Aqui usaremos o `RMSprop`; o pacote optim contém muitos outros algoritmos de otimização. O primeiro argumento para o construtor `RMSprop` informa ao otimizador quais tensores ele deve atualizar.</font>

In [5]:
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)


In [6]:
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    """
    Chamar a função step em um Otimizador para atualiza seus parâmetros:
    """
    optimizer.step()

99 38493.21484375
199 19101.02734375
299 8475.0986328125
399 3065.74365234375
499 901.3574829101562
599 325.8468933105469
699 208.62718200683594
799 152.314453125
899 108.38202667236328
999 73.01197814941406
1099 45.56995391845703
1199 26.44384765625
1299 15.201367378234863
1399 10.23926830291748
1499 8.95171070098877
1599 8.961365699768066
1699 8.817235946655273
1799 9.509202003479004
1899 8.864957809448242
1999 8.922170639038086


In [8]:
linear_layer = model[0]
linear_layer

Linear(in_features=3, out_features=1, bias=True)

In [9]:
linear_layer1 = model[1]
linear_layer1

Flatten(start_dim=0, end_dim=1)

In [10]:
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

Result: y = 0.0003386259777471423 + 0.857300341129303 x + 0.0003473146352916956 x^2 + -0.092758409678936 x^3
